In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns

In [5]:
df_dp = pd.read_csv(os.path.realpath('data/dp_raw_new.csv'))
df_dp.head()

,user_id,question_id,question_title,question_text,answer_id,answer_text,answer_score,user_answer_count,user_sum_score,user_mean_score,user_median_score,sum_class,mean_class,median_class
0,2970947,20538970,Show that a parameter of a method serves as re...,Is there a way to show that a parameter of a m...,20539063,"Firstly, you could do this - \n\nThen you coul...",0,18,214,11.89,1.0,5,3,0
1,2970947,27131405,type-casting can be an example of adapter desi...,Adapter design pattern resolved the incompatib...,27131703,No. Since an adapter pattern requires Object i...,1,18,214,11.89,1.0,5,3,0
2,2970947,20233780,Object Pool Pattern in Java,So I've implemented my own Object Pool Pattern...,20234459,If your Object Pool is actually returning inst...,2,18,214,11.89,1.0,5,3,0
3,2970947,20252715,How to use the NoSuchElementException in Itera...,I wonder why the Iterator.next() throws a NoSu...,20252839,The might throw NoSuchElementException for a ...,1,18,214,11.89,1.0,5,3,0
4,2970947,19995247,proxy design pattern using reflection,I came across the code given below from the fo...,19995396,What does that mean? It creates a new Object (...,0,18,214,11.89,1.0,5,3,0


In [6]:
df_se = pd.read_csv(os.path.realpath('data/se_raw_new.csv'))
df_se.head()

,user_id,question_id,question_title,question_text,answer_id,answer_text,answer_score,user_answer_count,user_sum_score,user_mean_score,user_median_score,sum_class,mean_class,median_class
0,36884,37695,When is Java a good choice for web development?,When is Java a good choice for web development...,146526,"Java is a statically typed language, and is ch...",-1,21,73,3.48,2.0,3,1,1
1,36884,247084,Is reading+modifying existing code better than...,I'm an inexperienced programmer. I have been a...,247092,"In ""The Psychology of Computer Programming"" Ge...",7,21,73,3.48,2.0,3,1,1
2,36884,247151,Track performance numbers,I have a (large) software project in a git rep...,247162,I believe that the best approach for your case...,2,21,73,3.48,2.0,3,1,1
3,36884,208923,Any languages use the &#39;=&gt;&#39; and &#39...,I'm just curious if there are any programming ...,208925,About =>\nC# does.\nLook here: http://msdn.mic...,10,21,73,3.48,2.0,3,1,1
4,36884,143429,Rpc transferring data or Replication,I have services on 10 machines that stores dat...,143433,It depends on how much data you need to get an...,1,21,73,3.48,2.0,3,1,1


In [7]:
def print_crop_stats(df):
    CROP_RATIO = 0.25

    df_filtered = df[(df['user_answer_count'] >= 5)] # first filter answers by user_answer_count
    n_answers_filtered = len(df_filtered)
    df_users_filtered = df_filtered.groupby('user_id').max()
    n_users_filtered = len(df_users_filtered)

    n_answers_needed = int(n_answers_filtered*CROP_RATIO)  # then crop by crop_ratio

    scorings = ['user_median_score', 'user_mean_score', 'user_sum_score']

    for scoring in scorings:

        df_sorted = df_filtered.sort_values(scoring, ascending=False, inplace=False)  # sort desc
        df_credible = df_sorted[0:n_answers_needed]  # get top n
        df_sorted = df_filtered.sort_values(scoring, ascending=True, inplace=False)  # sort asc
        df_not_credible = df_sorted[0:n_answers_needed]  # get top n

        n_answers_credible = len(df_credible)
        n_answers_not_credible = len(df_not_credible)


        df_users_credible = df_credible.groupby('user_id')[[scoring]].max()
        df_users_not_credible = df_not_credible.groupby('user_id')[[scoring]].max()

        n_users_credible = len(df_users_credible[scoring])
        n_users_not_credible = len(df_users_not_credible[scoring])

        min_credible_user_score = min(df_users_credible[scoring])
        max_not_credible_user_score = max(df_users_not_credible[scoring])

        print('***** by {} *****'.format(scoring))
        print('num of credible answers: {} ({}% of {})'.format(
              n_answers_credible, round(n_answers_credible*100/n_answers_filtered), n_answers_filtered))
        print('num of credible users: {} ({}% of {})'.format(
              n_users_credible, round(n_users_credible*100/n_users_filtered), n_users_filtered))
        print('min score of credible users: {}'.format(min_credible_user_score))



        print('num of not credible answers: {} ({}% of {})'.format(
              n_answers_not_credible, round(n_answers_not_credible*100/n_answers_filtered), n_answers_filtered))
        print('num of not credible users: {} ({}% of {})'.format(
              n_users_not_credible, round(n_users_not_credible*100/n_users_filtered), n_users_filtered))
        print('max score of not credible users: {}'.format(max_not_credible_user_score))
        print('num of total answers: {}'.format(n_answers_credible + n_answers_not_credible))
        print('num of total users: {}\n\n'.format(n_users_credible + n_users_not_credible))


In [8]:
print_crop_stats(df_dp)

***** by user_median_score *****
num of credible answers: 5723 (25% of 22894)
num of credible users: 478 (25% of 1942)
min score of credible users: 2.0
num of not credible answers: 5723 (25% of 22894)
num of not credible users: 518 (27% of 1942)
max score of not credible users: 1.0
num of total answers: 11446
num of total users: 996


***** by user_mean_score *****
num of credible answers: 5723 (25% of 22894)
num of credible users: 430 (22% of 1942)
min score of credible users: 4.19
num of not credible answers: 5723 (25% of 22894)
num of not credible users: 568 (29% of 1942)
max score of not credible users: 1.35
num of total answers: 11446
num of total users: 998


***** by user_sum_score *****
num of credible answers: 5723 (25% of 22894)
num of credible users: 157 (8% of 1942)
min score of credible users: 118
num of not credible answers: 5723 (25% of 22894)
num of not credible users: 850 (44% of 1942)
max score of not credible users: 15
num of total answers: 11446
num of total users: 

In [9]:
print_crop_stats(df_se)

***** by user_median_score *****
num of credible answers: 30975 (25% of 123902)
num of credible users: 753 (20% of 3841)
min score of credible users: 4.0
num of not credible answers: 30975 (25% of 123902)
num of not credible users: 1435 (37% of 3841)
max score of not credible users: 2.0
num of total answers: 61950
num of total users: 2188


***** by user_mean_score *****
num of credible answers: 30975 (25% of 123902)
num of credible users: 702 (18% of 3841)
min score of credible users: 8.67
num of not credible answers: 30975 (25% of 123902)
num of not credible users: 1666 (43% of 3841)
max score of not credible users: 3.41
num of total answers: 61950
num of total users: 2368


***** by user_sum_score *****
num of credible answers: 30975 (25% of 123902)
num of credible users: 52 (1% of 3841)
min score of credible users: 2313
num of not credible answers: 30975 (25% of 123902)
num of not credible users: 2833 (74% of 3841)
max score of not credible users: 134
num of total answers: 61950
nu